In [37]:
import torch
import torch.nn as nn
import torch.optim as optims
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer
import torch.nn.functional as F
import torch.distributions as dist
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from transformers import DistilBertModel, DistilBertTokenizer

In [65]:
# input_path = '/kaggle/input/iu-xray-reports-preprocessed-sorted-df/iu_xray_reports_preprocessed_sorted_df.csv'
# output_dir = '/kaggle/working/'

input_test_path = '/home/raginivi/CS550_ASMT_MRSMK/filtered_test_data.csv'
input_train_path = '/home/raginivi/CS550_ASMT_MRSMK/filtered_train_data.csv'
output_path = '/home/raginivi/CS550_ASMT_MRSMK'

In [170]:
pip install torch --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 30.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 31.0 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.6 MB/s eta 0:00:000:00:0136m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.4 MB/s eta 0:00:00m eta 0:00:010:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 28.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 30.4 MB/s eta 0:00:000:00:010:00:0102
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 36.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 35.0 MB/s eta 0:00:00m eta 0:00:010:01:02
   ━━━━━━━━━━━━━━━━━━━━━

In [67]:
import os
import pandas as pd
# os.getcwd()

train_path = pd.read_csv(input_train_path)
test_path = pd.read_csv(input_test_path)

print(train_path.shape, test_path.shape)

(2705, 5) (763, 5)


In [69]:
!pip3 uninstall ipywidgets --yes
!pip3 install --upgrade ipywidgets

Found existing installation: ipywidgets 8.1.5
Uninstalling ipywidgets-8.1.5:
  Successfully uninstalled ipywidgets-8.1.5
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [70]:
!pip install transformers torch  # For Hugging Face
# !pip install sentence-transformers  # For Sentence Transformers
!pip install -U sentence-transformers


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [119]:
class SentenceEncoder(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=512, output_dim=512):
        super(SentenceEncoder, self).__init__()
        # Encoder network
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # Networks to generate mean and variance as mentioned in paper
        self.mean_layer = nn.Linear(hidden_dim, output_dim)
        self.logvar_layer = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        encoded = self.encoder(x)
        mean = self.mean_layer(encoded)
        logvar = self.logvar_layer(encoded)
        return mean, logvar
        
    def encode(self, x):
        """Get only the mean for inference"""
        mean, _ = self.forward(x)
        return mean

In [121]:
class SentenceBERT:
    def __init__(self):
        # Initialize the BERT tokenizer and model
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')

    def encode_reports(self, reports):
        embeddings = []
        for report in reports:
            # Tokenize and encode the report
            inputs = self.tokenizer(report, return_tensors='pt', padding=True, truncation=True)

            # Get BERT outputs
            with torch.no_grad():  # Disable gradient calculation for inference
                outputs = self.bert_model(**inputs)
                
            # Use the pooled output as the embedding
            embeddings.append(outputs.pooler_output)

        # Stack all embeddings into a single tensor
        return torch.stack(embeddings)


In [186]:
from tqdm import tqdm  # Import tqdm for progress bar

# Initialize models
sentence_bert = SentenceBERT()
sentence_encoder = SentenceEncoder()

# Parameters for batching
batch_size = 64
num_batches = (len(df) + batch_size - 1) // batch_size  # Calculate number of batches

# Initialize matrices to store embeddings
sbert_matrix = []
sentence_encoder_matrix = []

# Loop through batches with progress bar
for i in tqdm(range(num_batches), desc="Processing Batches", unit="batch"):
    batch_reports = df['findings'].iloc[i * batch_size:(i + 1) * batch_size].tolist()
    
    # Compute SBERT embeddings
    sbert_embeddings = sentence_bert.encode_reports(batch_reports)
    
    # Compute Sentence Encoder embeddings
    sentence_embeddings = sentence_encoder.encode(sbert_embeddings)
    
    # Store embeddings in the matrices
    sbert_matrix.append(sbert_embeddings)
    sentence_encoder_matrix.append(sentence_embeddings)

# Stack the matrices to create final output tensors
sbert_matrix = torch.cat(sbert_matrix, dim=0).squeeze(1)
sentence_encoder_matrix = torch.cat(sentence_encoder_matrix, dim=0).squeeze(1)

# Print shapes of the final matrices
print(f"SBERT Matrix Shape: {sbert_matrix.shape}")
print(f"Sentence Encoder Matrix Shape: {sentence_encoder_matrix.shape}")


Processing Batches: 100%|████████████████████| 43/43 [03:01<00:00,  4.22s/batch]

SBERT Matrix Shape: torch.Size([2705, 768])
Sentence Encoder Matrix Shape: torch.Size([2705, 512])


In [188]:
def kl_divergence_loss(mean1: torch.Tensor, logvar1: torch.Tensor, mean2: torch.Tensor, logvar2: torch.Tensor) -> torch.Tensor:
    normal1 = dist.Normal(mean1, torch.exp(0.5 * logvar1))  # Standard deviation is sqrt of variance
    normal2 = dist.Normal(mean2, torch.exp(0.5 * logvar2))
    kl_loss = dist.kl.kl_divergence(normal1, normal2).mean()
    return kl_loss

In [213]:
def train_embeddings(sentence_encoder, projection_layer, sentence_encoder_matrix, sbert_matrix, 
                    optimizer, batch_size, num_epochs):
    """
    Training loop with explicit memory management and graph cleanup
    """
    device = next(sentence_encoder.parameters()).device
    
    for epoch in range(num_epochs):
        total_loss = 0
        sentence_encoder.train()
        projection_layer.train()
        
        n_batches = sentence_encoder_matrix.size(0) // batch_size
        
        with tqdm(total=n_batches, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch") as pbar:
            for i in range(0, sentence_encoder_matrix.size(0), batch_size):
                try:
                    # Clear gradients
                    optimizer.zero_grad(set_to_none=True)  # More efficient than zero_grad()
                    
                    # Get batch data with detached copies
                    batch_embedding1 = sentence_encoder_matrix[i:i+batch_size].to(device).detach().clone()
                    batch_embedding2 = sbert_matrix[i:i+batch_size].to(device).detach().clone()
                    
                    # Project SBERT embeddings
                    with torch.set_grad_enabled(True):
                        batch_embedding2_projected = projection_layer(batch_embedding2)
                        
                        # Compute means and variances
                        mean1 = batch_embedding1.mean(dim=0, keepdim=True)
                        var1 = batch_embedding1.var(dim=0, keepdim=True, unbiased=False)
                        mean2 = batch_embedding2_projected.mean(dim=0, keepdim=True)
                        var2 = batch_embedding2_projected.var(dim=0, keepdim=True, unbiased=False)
                        
                        # KL divergence loss
                        kl_loss = kl_divergence_loss(mean1, var1, mean2, var2)
                        
                        # Similarity loss
                        similarities = F.cosine_similarity(batch_embedding1, batch_embedding2_projected, dim=1)
                        sim_loss = 1 - similarities.mean()
                        
                        # Combined loss
                        loss = kl_loss + sim_loss
                    
                    # Backward pass with no graph retention
                    loss.backward(retain_graph=False)
                    
                    # Clip gradients
                    torch.nn.utils.clip_grad_norm_(sentence_encoder.parameters(), max_norm=1.0)
                    torch.nn.utils.clip_grad_norm_(projection_layer.parameters(), max_norm=1.0)
                    
                    # Optimizer step
                    optimizer.step()
                    
                    # Update metrics
                    current_loss = loss.item()
                    total_loss += current_loss
                    
                    # Clean up tensors explicitly
                    del batch_embedding1, batch_embedding2, batch_embedding2_projected
                    del mean1, var1, mean2, var2
                    del kl_loss, similarities, sim_loss, loss
                    torch.cuda.empty_cache() if torch.cuda.is_available() else None
                    
                    # Update progress bar
                    pbar.update(1)
                    pbar.set_postfix(loss=f"{current_loss:.4f}")
                    
                except RuntimeError as e:
                    print(f"\nError in batch {i//batch_size}:")
                    print(str(e))
                    # Try to recover
                    torch.cuda.empty_cache() if torch.cuda.is_available() else None
                    continue
        
        # Compute average loss for the epoch
        avg_loss = total_loss / n_batches
        print(f"\nEpoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

# Set up training
batch_size = 64
num_epochs = 20

# Initialize optimizer with parameters from both models
optimizer = optim.Adam([
    {'params': sentence_encoder.parameters()},
    {'params': projection_layer.parameters()}
], lr=1e-3)

# Move models to the same device if using GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    sentence_encoder = sentence_encoder.to(device)
    projection_layer = projection_layer.to(device)
    sentence_encoder_matrix = sentence_encoder_matrix.to(device)
    sbert_matrix = sbert_matrix.to(device)
else:
    device = torch.device('cpu')

# Run training
train_embeddings(
    sentence_encoder=sentence_encoder,
    projection_layer=projection_layer,
    sentence_encoder_matrix=sentence_encoder_matrix,
    sbert_matrix=sbert_matrix,
    optimizer=optimizer,
    batch_size=batch_size,
    num_epochs=num_epochs
)

Epoch 1/20: 43batch [00:00, 122.05batch/s, loss=0.0630]                         



Epoch 1/20, Average Loss: 0.1346


Epoch 2/20: 43batch [00:00, 162.24batch/s, loss=0.0574]                         



Epoch 2/20, Average Loss: 0.0622


Epoch 3/20: 43batch [00:00, 126.77batch/s, loss=0.0562]                         


...


Epoch 18/20, Average Loss: 0.0593


Epoch 19/20: 43batch [00:00, 117.16batch/s, loss=0.0525]                        



Epoch 19/20, Average Loss: 0.0592


Epoch 20/20: 43batch [00:00, 201.05batch/s, loss=0.0524]                        


Epoch 20/20, Average Loss: 0.0591


In [ ]:
def compute_cosine_similarities(batch_embeddings: torch.Tensor, all_embeddings: torch.Tensor) -> torch.Tensor:
    # Normalize embeddings
    batch_norm = F.normalize(batch_embeddings, p=2, dim=1)
    all_norm = F.normalize(all_embeddings, p=2, dim=1)
    
    # Compute similarities
    similarities = torch.mm(batch_norm, all_norm.t())
    return similarities

# Compute similarity matrices
sbert_similarities = compute_cosine_similarities(sbert_matrix, sbert_matrix)
sentence_encoder_similarities = compute_cosine_similarities(sentence_encoder_matrix, sentence_encoder_matrix)

# Print similarity matrix shapes
print(f"SBERT Similarity Matrix Shape: {sbert_similarities.shape}")  # Should be [2705, 2705]
print(f"Sentence Encoder Similarity Matrix Shape: {sentence_encoder_similarities.shape}")  # Should be [2705, 2705]